In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_path = os.path.abspath("..")

sys.path.append(project_path)

In [2]:
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from src.utils import fetch_page_content
from src.scrap_infobox import download_unit_icon, extract_item_vals, extract_unit_data

In [3]:
url = "https://ageofempires.fandom.com/wiki/War_Wagon_(Age_of_Empires_III)"
html_content = fetch_page_content(url)
soup = bs(html_content, "html.parser")
infobox = soup.find('aside', class_='portable-infobox')

In [4]:
img_path = os.path.join(project_path, "images")
download_unit_icon(infobox, img_path)

Icono guardado como: 'war_wagon_de.png' en 'c:\Users\Jonathan Diaz\Documentos\Proyects\aoe3de_web_scraper\images'


In [5]:
blocks = infobox.find_all("section")

In [6]:
block = blocks[2]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[1]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")
for feat in item.children:
    if isinstance(feat, Tag) and "image" in (feat.get("class") or []):
        continue
    
    print("=========================================================================================")
    print("=========================================================================================")
    print("Type: ", type(feat).__name__)
    print("Name Tag: ", feat.name)
    # print("Class: ", feat_class)
    print("Text: ", feat.text)
    print()
    print("Content: ", feat)

Type:  NavigableString
Name Tag:  None
Text:  20% 

Content:  20% 
Type:  NavigableString
Name Tag:  None
Text:   Hand

Content:   Hand
Type:  Tag
Name Tag:  br
Text:  

Content:  <br/>


In [8]:
block = blocks[5]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[3]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")

extract_item_vals(item, "text", label)

'Same as ranged attack'

In [11]:
unit_data = extract_unit_data(infobox)

In [12]:
import json

print(json.dumps(unit_data, indent=4))

{
    "name": "War Wagon",
    "Information": {
        "Introduced in": "Age of Empires III",
        "Type": [
            "Cavalry",
            "Light ranged cavalry",
            "Ranged cavalry",
            "Gunpowder cavalry"
        ],
        "Civilization(s)": [
            "Germans",
            "John Black's Mercenaries (Act II: Ice)",
            "South Africa"
        ],
        "Age": [
            "Fortress Age",
            "Revolution (South Africa)"
        ],
        "Required Home City Card": "\"Wagon Warfare\" (South Africa)"
    },
    "Training": {
        "Trained at": [
            "Stable",
            "Fort",
            "Galleon",
            "Fluyt (South Africa)",
            "Steamer (Gran Colombia)"
        ],
        "Cost": {
            "Food": "150",
            "Coin": "150",
            "Population": "3"
        },
        "Train time": {
            "seconds": "60"
        }
    },
    "Statistics": {
        "Hit points": "500",
        "Resist